# Computer Vision Learner

[`vision.learner`](/vision.learner.html#vision.learner) is the module that defines the `Conv_Learner` class, to easily get a model suitable for transfer learning.

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.vision import *
from fastai import *
from fastai.docs import *

## Transfer learning:

Transfer learning is a technique where you use a model trained on a very large dataset (usually [ImageNet](http://image-net.org/) in computer vision) and then adapt it to your own dataset. The idea is that it has learned to recognize many features on all of this data, and that you will benefit from this knowledge, especially if your dataset is small, compared to starting from a randomly initiliazed model. It has been proved in [this article](https://arxiv.org/abs/1805.08974) on a wide range of tasks that transfer learning nearly always give better results.

In practice, you need to change the last part of your model to be adapted to your own number of classes. Most convolutional models end with a few linear layers (a part will call head). The last convolutional layer will have analyzed features in the image that went through the model, and the job of the head is to convert those in predictions for each of our classes. In transfer learning we will keep all the convolutional layers (called the body or the backbone of the model) with their weights pretrained on ImageNet but will define a new head initiliazed randomly.

Then we will train the model we obtain in two phases: first we freeze the body weights and only train the head (to convert those analyzed features into predictions for our own data), then we unfreeze the layers of the backbone (gradually if necessary) and fine-tune the whole model (possily using differential learning rates).

The [`ConvLearner`](/vision.learner.html#ConvLearner) class helps you to automatically get a pretrained model from a given architecture with a custom head that is suitable for your data.

In [ ]:
show_doc(ConvLearner, doc_string=False)

## <a id=ConvLearner></a>class `ConvLearner`
`ConvLearner`(`data`:[`DataBunch`](/data.html#DataBunch), `arch`:`Callable`, `cut`:`Union`\[`int`, `Callable`\]=`None`, `pretrained`:`bool`=`True`, `lin_ftrs`:`Optional`\[`Collection`\[`int`\]\]=`None`, `ps`:`Union`\[`float`, `Collection`\[`float`\]\]=`0.5`, `custom_head`:`Optional`\[[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)\]=`None`, `split_on`:`Union`\[`Callable`, `Collection`\[`Collection`\[[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)\]\], `NoneType`\]=`None`, `kwargs`:`Any`) :: [`Learner`](/basic_train.html#Learner)
<a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/vision/learner.py#L44">[source]</a>

This class creates a [[[`Learner`](/basic_train.html#Learner)](/basic_train.html#Learner)](/basic_train.html#Learner) object from the [`data`](/vision.data.html#vision.data) object and model inferred from it with the backbone given in `arch`. Specifically, it will cut the model defined by `arch` (randomly initialized if `pretrained` is False) at the last convolutional layer by default (or as defined in `cut`, see below) and add:
- an [`AdaptiveConcatPool2d`](/layers.html#AdaptiveConcatPool2d) layer,
- a [`Flatten`](/layers.html#Flatten) layer,
- blocks of \[`nn.BatchNorm1d`, `nn.Dropout`, `nn.Linear`, `nn.ReLU`\] layers.

The blocks are defined by the `lin_ftrs` and `ps` arguments. Specifically, the first block will have a number of inputs inferred from the backbone `arch` and the last one will have a number of outputs equal to `data.c` (which contains the number of classes of the data) and the intermediate blocks have a number of inputs/outputs determined by `lin_frs` (of course a block has a number of inputs equal to the number of outputs of the previous block). The dropouts all have a the same value `ps` if you pass a float, or the corresponding values if you pass a list. Default is to have an intermediate hidden size of 512 (which makes two blocks model\_activation -> 512 -> n\_classes).

Note that the very last block doesn't have a `nn.ReLU` activation, to allow you to use any final activation you want (generally included in the loss function in pytorch). Also, the backbone will be frozen if you choose `pretrained`=True (so only the head will train if you call [`fit`](/basic_train.html#fit)) so that you can immediately start phase one of training as described above.

Alternatively, you can define your own `custom_head` to put on top of the backbone. If you want to specify where to split `arch` you should so in the argument `cut` which can either be the index of a specific layer (the result will not include that layer) or a function that, when passed the model, will return the backbone you want.

The final model obtained by stacking the backbone and the head (custom or defined as we saw) is then separated in groups for gradual unfreezeing or differential learning rates. You can specify of to split the backbone in groups with the optional argument `split_on` (should be a function that returns those groups when given the backbone).  

The `kwargs` will be passed on to [[[`Learner`](/basic_train.html#Learner)](/basic_train.html#Learner)](/basic_train.html#Learner), so you can put here anything that [[[`Learner`](/basic_train.html#Learner)](/basic_train.html#Learner)](/basic_train.html#Learner) will accept ([`metrics`](/metrics.html#metrics), `loss_fn`, `opt_fn`...)

In [ ]:
#Uncomment to extract the MNIST sample dataset
#untar_mnist()

In [ ]:
data = image_data_from_folder('../data/mnist_sample/', ds_tfms=get_transforms(do_flip=False), size=32)

In [ ]:
learner = ConvLearner(data, tvm.resnet34, metrics=[accuracy])
learner.fit_one_cycle(1,1e-3)

Total time: 00:14
epoch  train loss  valid loss  accuracy
0      0.113352    0.045578    0.980864  (00:14)



## Customize to your model

If you want to create a [`ConvLearner`](/vision.learner.html#ConvLearner) customized to your own architecture, you'll probably need to use the following helper functions. Note that you can include your own models default `cut` and `split_on` by adding it to the dictionary `model_meta`. The key should be your model and the value should be a dictionary with the keys `cut` and `split_on` (see the source code for examples).

In [ ]:
show_doc(create_body)

#### <a id=create_body></a>`create_body`
`create_body`(`model`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), `cut`:`Optional`\[`int`\]=`None`, `body_fn`:`Callable`\[[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module), [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)\]=`None`)


Cut off the body of a typically pretrained `model` at `cut` or as specified by `body_fn`. <a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/vision/learner.py#L9">[source]</a>

In [ ]:
show_doc(create_head, doc_string=False)

#### <a id=create_head></a>`create_head`
`create_head`(`nf`:`int`, `nc`:`int`, `lin_ftrs`:`Optional`\[`Collection`\[`int`\]\]=`None`, `ps`:`Union`\[`float`, `Collection`\[`float`\]\]=`0.5`)
<a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/vision/learner.py#L19">[source]</a>

Model head that takes `nf` features, runs through `lin_ftrs`, and endw with `nc` classes. `ps` represent the probability of the dropouts, and can be a single float or a list for each layer.

In [ ]:
show_doc(num_features)

#### <a id=num_features></a>`num_features`
`num_features`(`m`:[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)) -> `int`


Return the number of output features for a `model`. <a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/vision/learner.py#L14">[source]</a>